In [25]:
!pip install datasets
!pip install transformers
!pip install torch


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [6]:
# Run this cell to import or install the Data Wrangler widget to show automatic visualization and generate code to fix data quality issues

try:
    import sagemaker_datawrangler
except ImportError:
    !pip install --upgrade sagemaker-datawrangler
    import sagemaker_datawrangler

# Display Pandas DataFrame to view the widget: df, display(df), df.sample()... 

In [7]:
import sagemaker.huggingface

In [8]:
from sagemaker import get_execution_role #this contains the permissions to access the bucket
import boto3 #aws python client
import pandas as pd
s3 = boto3.client('s3') #connect to S3
bucketName = "cs539-e2023-149b5d57-81a4-4e31-a476-c81e7e07dc33"
training_dataset = 'training_reviews_clean_trimmed.tsv'
live_dataset = 'yelp_academic_dataset_review_trimmed.tsv'

In [18]:
response = s3.get_object(Bucket=bucketName, Key='datasets/tsv/'+training_dataset)
df = pd.read_csv(response.get("Body"), 
                sep='\t')
df

                                  review_id  \
0      ab58df3f-a1b3-40a1-af03-724c2db89f2a   
1      42b8ed9b-3e0e-4df9-acd2-266828a84abc   
2      fff42ce0-1e8d-4560-b961-97e8ef86e495   
3      774cd776-8a68-42f6-a5bb-c1b7f3ff7396   
4      f91991a1-c0b5-4ee4-8c6f-94d076c02420   
...                                     ...   
67014  60cf9b0a-a021-4443-a977-d839ea01ec4e   
67015  fe3105ef-b98b-4f32-893a-8c33929c09d9   
67016  0b008579-68af-45eb-8469-25b825c70768   
67017  0ce3171f-f0ec-4345-a39d-c719d7c40a90   
67018  4a9c04ce-d57c-4b2b-8c24-779c20347066   

                                                    text  flagged  
0      Unlike Next which we'd eaten at the previous n...        0  
1      Probably one of the best meals I've had ever. ...        0  
2      Service was impeccable. Experience and present...        0  
3      The problem with places like this given the ex...        0  
4      I have no idea how to write my review  dining ...        0  
...                        

In [19]:
print(len(df))

67019


In [11]:
import time
import multiprocessing
from transformers import AutoTokenizer, BertForSequenceClassification

MODEL_NAME = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [22]:
import io
import json
def tokenizeFunc(reviewText):
    output = tokenizer(str(reviewText), max_length=256, truncation=True, padding='max_length', add_special_tokens=True, return_token_type_ids=False)
    return output;


def threadRunner(df_chunk, chunk_index, write_to_disk=True):
    
    #create output structure and copy constant values
    output = pd.DataFrame(columns={'review_id':str,'token_ids':list, 'attention_mask':list, 'flagged':int})
    #output = pd.DataFrame(columns={'review_id':str,'token_ids':list, 'attention_mask':list})
    output['review_id'] = df_chunk['review_id']
    output['flagged'] = df_chunk['flagged']
    
    #tokenize the text column and store the token_ids and attention mask in the output
    for i in range(len(df_chunk)):
        encoding = tokenizeFunc(df_chunk['text'][i])
        output.at[i, 'token_ids'] = encoding.input_ids
        output.at[i, 'attention_mask'] = encoding.attention_mask
    
    if write_to_disk:
        chunk_id = "{:03d}".format(chunk_index)
        writeKey = 'tokenized/tsv/training_reviews/chunk_' + str(chunk_id) + '.tsv'
        with io.StringIO() as tsv_buffer: #buffer to write the data
            output.to_csv(tsv_buffer, index=False, sep='\t')
            s3.put_object(Bucket=bucketName, Key=writeKey, Body=tsv_buffer.getvalue())
    return output

In [23]:
from collections import deque

#prep data for multi-threading
chunk_size = 10000 #num rows per chunk
chunks = []
for i in range(0, len(df), chunk_size):
    chunk = df[i:i+chunk_size]
    #reset row index
    chunk = chunk.reset_index(drop=True)
    chunks.append(chunk)
print(len(chunks))

#configure threading
num_threads = multiprocessing.cpu_count()
processes = []

7


In [24]:
for i in range(len(chunks)):
    p = multiprocessing.Process(target=threadRunner, args=(chunks[i], i))
    processes.append(p)
    p.start()
    
for p in processes:
    p.join()